# Proyecto Final – Ciencia de Datos

### Predicción de Ventas Globales de Videojuegos

Este proyecto tiene como objetivo predecir las ventas globales (`Global_Sales`) de videojuegos utilizando técnicas de ciencia de datos. El análisis se basa en un dataset histórico con información sobre títulos, plataformas, géneros, puntuaciones de críticos y usuarios, entre otras variables. 

A lo largo del proyecto se aplicarán los siguientes pasos:
1. Limpieza y preprocesamiento de los datos
2. Selección de características (feature selection)
3. Entrenamiento de un modelo de regresión
4. Evaluación del modelo mediante métricas apropiadas
5. Conclusiones basadas en los resultados obtenidos


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
# Cargar datos desde GitHub
url = "https://raw.githubusercontent.com/MarcosRomano13/proyecto-ciencia-datos-videojuegos/main/Dataset_final.csv"
df = pd.read_csv(url)

# Normalizar columnas
df.columns = df.columns.str.strip().str.replace(" ", "_")

# Eliminar filas con valores nulos importantes
df = df.dropna(subset=["Global_Sales", "Critic_Score", "User_Score", "Genre", "Platform", "Year_of_Release"])

# Convertir User_Score a numérico
df["User_Score"] = pd.to_numeric(df["User_Score"], errors="coerce")
df = df.dropna(subset=["User_Score"])

# Codificar variables categóricas
from sklearn.preprocessing import LabelEncoder
le_genre = LabelEncoder()
le_platform = LabelEncoder()
df["Genre"] = le_genre.fit_transform(df["Genre"])
df["Platform"] = le_platform.fit_transform(df["Platform"])


In [ ]:
# Selección de características con SelectKBest
X = df[["Critic_Score", "User_Score", "Genre", "Platform", "Year_of_Release"]]
y = df["Global_Sales"]

selector = SelectKBest(score_func=f_regression, k=5)
X_new = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("Variables seleccionadas:", list(selected_features))


In [ ]:
# Separar los datos
X_selected = df[selected_features]
y = df["Global_Sales"]

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)


In [ ]:
# Predicción y métricas
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Coeficiente de determinación (R2): {r2:.2f}")


### Conclusiones

El modelo de regresión fue entrenado para predecir las ventas globales de videojuegos utilizando un conjunto reducido de variables seleccionadas por `SelectKBest`. Las variables más influyentes según esta técnica fueron:

- Puntuación de críticos (`Critic_Score`)
- Puntuación de usuarios (`User_Score`)
- Género (`Genre`)
- Plataforma (`Platform`)
- Año de lanzamiento (`Year_of_Release`)

Los resultados muestran un R² aceptable, lo cual indica que el modelo logra capturar parte de la variabilidad en las ventas globales. Sin embargo, como se trata de un fenómeno multifactorial influido por aspectos como marketing, contexto histórico y popularidad de franquicias, el modelo puede seguir mejorándose.

Posibles mejoras:
- Hacer tuning de hiperparámetros del modelo
- Probar otros algoritmos como XGBoost o LightGBM
- Incorporar nuevas variables si estuvieran disponibles
